# Data Cleaning & EDA
*Author: [Douglas Strodtman](http://linkedin.com/in/dstrodtman/)*



Data cleaning and exploratory data analysis often go hand in hand. 
- Without examining our data, it's difficult to know whether or not there are errors in it. 
- Without cleaning our data, our aggregate statistics may be skewed by errant data.
The interplay of these processes is often very cyclical. For a data science workflow, these steps are essential to help us understand the nature of our data and ensure that we haven't injected or propogated unnecessary noise to our modeling algorithm. Oftentimes we will find ourselves circling back to data cleaning and EDA after modeling when we are dissatisfied with results.

**No matter your goals working with data, becoming proficient with cleaning and EDA is amongst the most important skills you can learn.**

## Skills Covered
1. Module import
1. Data import
1. Previewing Data
1. Renaming Columns
1. Masking
1. Reindexing
1. Summary Statistics
1. `groupby` and Aggregation
1. Pivot Tables
1. Missing data
    - Finding missing values
    - Imputing missing data
1. Data export

## Key Objectives

Our walkthrough will focus on data from the years 2017 and 2018. By the end of this lesson, you'll be able to answer the following questions (which will be the focus of the accompanying lab):

- Which department had the most line item entries each year?
- Which department had the highest total expenditures each year?
- Which fund had the highest budget allocation each year?
- What percentage of money from the general fund was allocated to different departments each year?
- Which departments saw the largest budget increase and decrease from 2017 to 2018?

## Module Import
Start off by importing pandas.

## Data Import
Load the full data. Use a relative path so that your code will be robust.

To see all the data files that were included with this lesson, run the following cell:

Import this to the variable `all_data`.

## Preview Data
Look at the first 5 rows of your data to see how it loaded.

While our default options appear to have successfully loaded the data, we have column names that are all caps and contain spaces. Let's fix this before moving forward.

## Renaming Columns

As long as our column names are only letters, numbers, and underscores, we can also use a dot notation to access Series. In addition, this format will work accross almost all parts of your data workflow, and is especially friendly to SQL.

Let's start by looking at all of our columns.

We're aiming for `snake_case` here, which means we'll want only lowercase letters and underscores.

Let's start by just saving our lowercase strings to a new variable, `columns_clean`.

As a next step, let's just replace the hyphens, overwriting our variable.

Finally, we can replace our spaces with underscores as well.

Because we've maintained the order of our columns, we can safely overwrite the original columns in our DataFrame.

Preview the first 3 rows to see that this worked.

## Masking

We're only interested in data from 2017 and 2018. Let's set up a unique mask for each of these years.

To do this, we'll just do a check for equality on our `budget_fiscal_year`.

We can now put these masks back into our DataFrame to look at only those rows for each year. Let's do this for each year and check the `shape` attribute so we can see how many rows we're selecting.

We can also use the bitwise `or` operator `|` to select all those rows where either of these conditions are true. The number of rows here should equal 7246.

Because we know that this is the data we wish to work with for the remainder of our exploration, let's save this out to a new DataFrame `df`.

And let's take a sample of 10 rows to do a quick check that we haven't included any data from other years.

## Reset Index

You'll note in the preview above that our indices are quite high. This index is not especially informative (it was generated by Pandas automatically upon import).

Personally, when my index doesn't correspond to a primary key, I prefer to work with a serial index starting at 0.

This method is also helpful for returning columns that you've used in a `groupby` statement back into your main DataFrame (more on this later).

Make sure to set the argument `drop=True` if you want to discard your old index (here, we desire this functionality).

In addition, once you've checked that your code is working, you should set `inplace=True` to persist these changes in your `df`.

## Summary Stats

We've already looked at the shape of our data, but let's check out our `info` to see the types and make note of any missing values.

And we can look at our overall numeric summary statistics. (Don't forget to transpose to make these easy to read).

Is there anything of value you note here? Do these numbers provide insight into any of the questions we originally sought to answers?

### Be careful!

Summary statistics can be misleading. The canonical example of this is [Ancombe's Quartet](https://en.wikipedia.org/wiki/Anscombe%27s_quartet)

**NOTE**: The following is taken directly from Wikipedia:

![Ancombe's Quartet](../images/anscombes_quartet.png)

| Property | Value | Accuracy |
| --- | --- | --- |
| Mean of $x$ | 9 | exact |
| Sample variance of $x$ | 11 | exact |
| Mean of $y$ | 7.50 | to 2 decimal places |
| Sample variance of $y$ | 4.125 | ±0.003 |
| Correlation between $x$ and $y$ | 0.816 | to 3 decimal places |
| Linear regression line | $y$&nbsp;=&nbsp;3.00&nbsp;+&nbsp;0.500$x$ | to 2 and 3 decimal places, respectively |
| Coefficient of determination of the linear regression | 0.67 | to 2 decimal places |

**While we can generally rely on summary statistics to give a good overview of our data, we should always do further data exploration before using summary statistics to support our claims.** Next week's lesson will focus exclusively on visualizing our data; data visualization is useful both for conveying findings and visually confirmation of mathematically-derived findings.

## `groupby` and Aggregation

We're not actually interested in aggregate statistics calculated over the entire column. Rather, we want to identify groups.

When using `groupby`, you'll need to also apply an aggregation method. Some useful aggregation methods include:

| method | function |
| --- | --- |
| `.count` | Returns the count of total rows that have been grouped together. |
| `.sum` | Returns the sum of all the rows in each group. |
| `.mean` | Returns the average of all the rows in each group. |

Let's start by just grouping by our `budget_fiscal_year` and calculating the mean. Transpose the result for easier interpretation.

We can also use `value_counts` and `describe` with `groupby`, but I'd recommend you limit these to a single column.

Let's use `describe` on our `total_budget` grouped by `budget_fiscal_year`.

All let's look at the `value_counts` of our `department_name` when grouped by year.

It's difficult to garner any insights from this preview.

## `.pivot_table`

Instead, we'll create a [pivot table](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html).

In my experience, power users of Excel think in pivot tables, whereas folks coming to data from a more programmatic background can struggle with the concept.

I think of it as allowing you to find the unique intersections of two different GROUP BY statements, and then identify an additional column to aggregate over.

Here's a break down of the arguments:

| arg | function |
| --- | --- |
| `values` | A column that will be aggregated |
| `index` | A column or list of columns; unique values will become the index of the resultant DataFrame |
| `columns` | A column or list of columns; unique values will become the columns of the resultant DataFrame |
| `aggfunc` | A aggregate function or list of aggregate functions that will be applied to the specified `values` column |

**Note**: There are additional layers of complex functionality available in this method, which is extremely powerful for data exploration.

Here, we'll return the `count` and `sum` of the `total_budget` with `department_name` as our index and `budget_fiscal_year` as our columns.

We're getting very close to being able to having all the tools we need to answer the questions we posed at the beginning of the lesson. However, there's still an elephant in the room...

## Missing Data

What _should_ we do about the missing values in our data?

This is a difficult question to answer. Unless you are reasonably confident that you can find the true value for a missing data point, you should always be careful when imputing a value. Without going too far into this, a few concerns with data imputation include:

1. Changes to distributions
1. Reduction in variance
1. Obfuscation of meaningful nulls
1. Data is "made up"

Many modeling techniques will require that you deal with all null values before moving forward, so there may be times that you have to impute missing values. A few common approaches include using:

1. The mean, median, or mode
1. A random value selected from the distribution of values in the sample
1. A placeholder to indicate missingness (e.g. -1, 999999, '?')

Each of these is imperfect, and in all cases, it's **imperative** to clearly indicate that you've edited missing values if you're going to store this data for later analysis. (Imagine coming across a dataset with a mostly normal distribution but a huge spike of values right at the median. How would you implicitly know whether this data were real or the result of data cleaning?)

That being said, let's go ahead look at the total number of missing values in each column to see if we can derive a plan of attack.

The `isna` method returns a boolean list that we can `sum` to get these counts.

I propose that the missingness represented by our 3 columns is unlikely to be random. Let's examine each column indepedently before making any decisions.

### Account Name

We'll start with `account_name`, which has the fewest nulls.

Let's begin by creating a mask of each row that has a missing value here.

We can then use this to review the values present in these rows.

My thought would be to see whether or not we can identify with cetainty the `account_name` by looking at other rows with the same `account` code.

We can use `isin` to find the rows that match here.

By selecting only those columns we're interested in and sorting them, we can quickly see that in these 4 cases, it's probably safe to impute the account names used in other instances of the account code.

While in this case our data were small enoug that we could visually review this, let's work out a way to do this programmatically.

The `dropna` method will, by default, drop rows that contain any nulls. **These changes will only persist if you use the `inplace=True` keyword argument.** Let's start building up our argument by again applying our mask, selecting our columns of interest, and dropping those rows containing nulls.

To get only those rows that are distinct, we can use `drop_duplicates`. Again, these changes won't persist unless we use the `inplace=True` argument.

Now we can clearly see that we have only one `account_name` for each `account`.

Calling `values` on the previous command will return an array.

Which we can cast as a dictionary to make our `account` the keys and the `account_name` the values.

Now when we `map` this back our `account` column for those missing rows, we'll return our `account_name`.

Which we can assign back to our DataFrame with `.loc`.

### Total Expenditures

If you recall from our first lab, the missingness in our total expenditures can't be easily calculated.

Let's create a `exp_null` mask so we can explore this feature.

Since we know we have a lot of rows here, let's just look at the first 10.

These first 10 rows have many zero values for the numeric columns, and also have identical values for the `budget_change_amount`, `total_budget`, and `budget_uncommitted_amount`. Let's explore how commonly these observations are true in the rest of the data.

Just for comparison, let's look at how commonly this amount is zero when expenditures aren't null.

Here we see that visual inspection of a sample led us to a spurious hypothesis. Indeed, we can remember from our earlier investigation of our summary statistics that many of our numeric fields have many 0 values. Let's abandon further investigation of zeroes for now.

Instead, let's build logic to investigate `budget_change_amount`, `total_budget`, and `budget_uncommitted_amount`.

Given that not all of our observations have 0 for the `adopted_budget_amount`, it follows that `budget_change_amount` and `total_budget` won't be equal in many of our rows.

We do see, however, that our uncommited budget is equal to our total budget in most of our data. If you recall, we should actually also be including the `encumbrance_amount` here.

This is very nearly every row. (We could push further and check for floating point errors in our calculations, but we'll skip this for now).

Based on our observations, do you think it's safe to impute `0` into our `total_expenditures` column for missing values?

Remember, most of our aggregate calculations in Pandas will ignore null values by default. If we impute something (whether it's zero, the mean, a numeric placeholder, or a random value), these values will factor into any future summary statistics. **I would recommend, whenever possible, that you avoid imputation until you have completed all of your EDA.**

### Account Group Name

This column had the highest number of nulls in our entire dataset. Let's look at the percentage.

With that many nulls, let's see what our `value_counts` are for this field.

The nature and distribution of these labels suggests that they are optional tags that provide additional context to line items. As such, the best we can likely due in order to eliminate the null values, is to use a filler string like `"UNSPECIFIED"`.

We can use `fillna` with the argument `inplace=True` to persist these changes to our data.

## Data Export

Saving data in Pandas is just as easy as loading data. Here, we'll save our data back to our data directory with the name `clean1718.csv` using the `to_csv` method. Because our numeric index is not meaningful here, we can pass the keyword argument `index=False`.